In [1]:
import requests
from bs4 import BeautifulSoup
import time
import numpy
import pandas as pd

/Users/leechinshan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Scrape Event Page

In [2]:
## Store in the list
base_url = 'https://visitseattle.org/events/page/'
detail_links = [] 

for i in range(1, 42):
    url = base_url + str(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    selector = 'div.search-result-preview > div > h3 > a'
    detail_eles = soup.select(selector)
    # detail_list.append([x['href'] for x in detail_eles])
    for ele in detail_eles:
        detail_links.append(ele['href'])

In [3]:
# save the detail_links into pickle
import pickle
with open('detail_links.pkl', 'wb') as f:
    pickle.dump(detail_links, f)
# read the pickle
with open('detail_links.pkl', 'rb') as f:
    detail_links = pickle.load(f)

In [4]:
## Print out the list
for link in detail_links:
    print(link)

https://visitseattle.org/events/6th-annual-sami-film-festival/
https://visitseattle.org/events/boldt-at-50/
https://visitseattle.org/events/an-evening-with-lucia-micarelli/
https://visitseattle.org/events/dylan-leblanc/
https://visitseattle.org/events/eliades-ochoa/
https://visitseattle.org/events/k-wayne-yang/
https://visitseattle.org/events/an-evening-with-lucia-micarelli-a-valentines-celebration/
https://visitseattle.org/events/juanes/
https://visitseattle.org/events/black-dogs/
https://visitseattle.org/events/black-history-month-keynote-with-dr-doretha-williams/
https://visitseattle.org/events/hear-me-talkin-to-you-womxn-and-blues/
https://visitseattle.org/events/maddie-zahm/
https://visitseattle.org/events/my-dinner-with-sam-aaron-verzosa/
https://visitseattle.org/events/olympia-music-history-project/
https://visitseattle.org/events/radiolab/
https://visitseattle.org/events/silversun-pickups/
https://visitseattle.org/events/the-brodies/
https://visitseattle.org/events/cory-wong-wi

## Scrape Detail Page

In [5]:
event_list = []

for event in detail_links:
    res = requests.get(event)
    soup = BeautifulSoup(res.text, 'html.parser')
    name_selector = '.medium-6.columns.event-top > h1'
    date_selector = '.medium-6.columns.event-top > h4 > span:nth-child(1)'
    location_selector = '.medium-6.columns.event-top > h4 > span:nth-child(2)'
    type_selector = '.medium-6.columns.event-top > a:nth-child(3)'
    region_selector = '.medium-6.columns.event-top > a:nth-child(4)'

    name = soup.select(name_selector)
    date = soup.select(date_selector)
    location = soup.select(location_selector)
    Type = soup.select(type_selector)
    region = soup.select(region_selector)
    event_list.append({'Name': name[0].text, 'Date': date[0].text, 'Location': location[0].text.strip(), 'Type': Type[0].text, 'Region': region[0].text})
    time.sleep(1)

df = pd.DataFrame(event_list)
df.to_csv('event_data.csv', index=False)

    

In [6]:
# save the event_list into pickle
with open('event_list.pkl', 'wb') as f:
    pickle.dump(event_list, f)
# read the pickle
with open('event_list.pkl', 'rb') as f:
    event_list = pickle.load(f)

## Weather API

In [7]:
Location_list = [event['Location'] for event in event_list]

In [8]:
map_url = "https://nominatim.openstreetmap.org/search"
weather_list = []

for i in range(len(Location_list)):
    location_query = f'{Location_list[i]}, Seattle, WA'
    query_params = {
        "q": location_query,
        "format": "jsonv2"
    }
    res = requests.get(map_url, params=query_params)
    map_dict = res.json()
    if map_dict == []:
        weather_list.append({'Location': Location_list[i], 'Day': 'No data', 'Temperature': 'No data', 'ShortForecast': 'No data'})
        continue
    latitude = map_dict[0]['lat']
    longitude = map_dict[0]['lon']
    
    weather_url = "https://api.weather.gov/points/{},{}".format(latitude, longitude)
    res = requests.get(weather_url)
    point_dict = res.json()
    print(weather_url)

    if 'properties' not in point_dict:
        print('No weather data')
        weather_list.append({'Location': Location_list[i], 'Day': 'No data', 'Temperature': 'No data', 'ShortForecast': 'No data'})
        continue
    else:
        forcast_url = point_dict['properties']['forecast']
        res = requests.get(forcast_url)
        weather_dict = res.json()

    if 'properties' not in weather_dict:
        print(forcast_url)
        weather_list.append({'Location': Location_list[i], 'Day': 'No data', 'Temperature': 'No data', 'ShortForecast': 'No data'})
        continue
    
    weather = weather_dict['properties']['periods']
    for x in weather:
        if x['isDaytime'] == True:
            weather_list.append({'Location': Location_list[i], 'Day': x['name'], 'Temperature': x['temperature'], 'ShortForecast': x['shortForecast']})


df = pd.DataFrame(weather_list)
df.to_csv('weather_data.csv', index=False)
weather_list

https://api.weather.gov/points/47.6090191,-122.32993774092222
https://api.weather.gov/points/47.6148163,-122.339691
https://api.weather.gov/points/47.6138274,-122.3196284
https://api.weather.gov/points/47.6090191,-122.32993774092222
https://api.weather.gov/points/47.6148163,-122.339691
https://api.weather.gov/points/47.6615049,-122.332319
https://api.weather.gov/points/47.589155399999996,-122.30193399673912
https://api.weather.gov/points/47.5569023,-122.2842722
https://api.weather.gov/points/47.63029565,-122.31413540328062
https://api.weather.gov/points/47.6605211,-122.31156049949824
https://api.weather.gov/points/47.6090191,-122.32993774092222
https://api.weather.gov/points/47.6514631,-122.3517391
https://api.weather.gov/points/47.6514631,-122.3517391
https://api.weather.gov/points/47.5569023,-122.2842722
https://api.weather.gov/points/47.6523876,-122.3538541520202
https://api.weather.gov/points/47.6657081,-122.38280295082915
https://api.weather.gov/points/47.6138274,-122.3196284
http

[{'Location': 'Majestic Bay Theatres',
  'Day': 'No data',
  'Temperature': 'No data',
  'ShortForecast': 'No data'},
 {'Location': 'Town Hall Seattle',
  'Day': 'Monday',
  'Temperature': 48,
  'ShortForecast': 'Light Rain Likely'},
 {'Location': 'Town Hall Seattle',
  'Day': 'Tuesday',
  'Temperature': 48,
  'ShortForecast': 'Mostly Sunny'},
 {'Location': 'Town Hall Seattle',
  'Day': 'Wednesday',
  'Temperature': 46,
  'ShortForecast': 'Chance Light Rain'},
 {'Location': 'Town Hall Seattle',
  'Day': 'Thursday',
  'Temperature': 45,
  'ShortForecast': 'Chance Light Rain'},
 {'Location': 'Town Hall Seattle',
  'Day': 'Friday',
  'Temperature': 47,
  'ShortForecast': 'Chance Light Rain'},
 {'Location': 'Town Hall Seattle',
  'Day': 'Saturday',
  'Temperature': 48,
  'ShortForecast': 'Chance Light Rain'},
 {'Location': 'Town Hall Seattle',
  'Day': 'Sunday',
  'Temperature': 51,
  'ShortForecast': 'Slight Chance Light Rain'},
 {'Location': "Dimitriou's Jazz Alley",
  'Day': 'Monday',
 